In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import geopandas as gpd
import fiona
from sqlalchemy import create_engine, URL
import psycopg2
from getpass import getpass

pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 10)

In [ ]:
psw = getpass("tbadie's password")
# psw = psw.replace('@', '%40')

url_agsdev_agsprodcopy = URL.create(
    "postgresql+psycopg2",
    username = "tbadie",
    password = psw,
    host = "147.100.179.208",
    port = '5438',
    database = "entrepot_20240923",
)
del(psw)

connection = create_engine(url_agsdev_agsprodcopy)

In [ ]:
# ------------------ #
# IMPORT DES DONNÉES #
# ------------------ #
ENTREPOT_PATH = '~/Bureau/Datagrosyst/data_entrepot_&_outils/'
EXTERN_DATA_PATH = '/home/administrateur/Bureau/Datagrosyst/data_entrepot_&_outils/referentiels/geoVec/'

df = {}

def import_df(df_name, path_data, sep, index_col=None, encoding = 'utf-8', geojson=False):
    if geojson == False :
        df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False, encoding = encoding).replace({'\r\n': '\n'}, regex=True)
    if geojson == True :
        df[df_name] = gpd.read_file(path_data+df_name+'.json', encoding = encoding)

def import_dfs(df_names, path_data, sep = ',', index_col=None, geojson=False, verbose=False, encoding = 'utf-8'):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col, geojson=geojson, encoding = encoding)

# manque les id pour merge à reflocation !!!
# tables_entrepot = [
#     'domaine'
# ]

# ref des communes avec toute les infos insee, créer à partir de 'create_geographic_ref_w_vineyard_basin_DG.R'
# le plus du script : ajout des ancienne région et BASSIN VITICOLES !
tables_externe = [
    'referentiel_geographique_fr_esr_unique' 
    ]

# on en aura besoin pour reocuper avec les mailles safran
geojson_extern = [
    'a-com2022',
    'geoVec_arrond',
    'geoVec_dep',
    'geoVec_reg',
    'geoVec_oldreg',
    'geoVec_viti'
]

# import des données de l'entrepôt avec la colonne 'id' en index 
# import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',', index_col='id', verbose=False)

# import des données externes
import_dfs(tables_externe, EXTERN_DATA_PATH, sep = ',', verbose=False, encoding='latin-1')

# import des données externe geojson
import_dfs(geojson_extern, EXTERN_DATA_PATH, verbose=False, geojson=True)

domain = pd.read_sql("SELECT * FROM domain", connection)
refloc = pd.read_sql("SELECT * FROM reflocation", connection)
esr = df['referentiel_geographique_fr_esr_unique']
esr = esr[['codeinsee','Municipality_code','Region_code_name','Department_code_name','Arrondissement_code_name','Intercomm_code_name','Municipality_name','Region_old','Vineyard_basin','Region_name']]

safran = gpd.read_file(EXTERN_DATA_PATH+'safran.gpkg/'+'safran.gpkg')

In [ ]:
domain = domain[['topiaid','location','campaign','active']]
refloc = refloc[refloc.columns.difference(['topiaversion','topiacreatedate','refcountry'])]
domain = domain.rename(columns={'topiaid': 'domain_id', 'campaign': 'year', 'active': 'active_domain'})
refloc = refloc.rename(columns={'topiaid': 'location', 'active': 'active_refloc'})

fdf = pd.merge(domain, refloc, on='location', how='left')
fdf = fdf.merge(esr, on=['codeinsee'], how='left')

In [ ]:
# Joindre les localisation agrosyst (communes) et mailles safran avec geopandas

# Créer un df des communes métropolitaines avec crs(=epsg) = 4326
com = df['a-com2022']
metrocom = com[~(com['dep'].str.match(r'97+') | com['dep'].str.match(r'98+'))]
metrocom = metrocom[['codgeo','geometry']].rename(columns={'codgeo': 'codeinsee'})
# on crée les centroid des communes, il faut passer en epsg de projection avant de créer les centroid (3857) puis remettre en 4326
metrocom = metrocom.assign(centroid = metrocom.to_crs(3857).centroid.to_crs(4326))

# m = metrocom.set_geometry('geometry').explore(name="Polygons")
# m = metrocom.set_geometry('centroid').explore(m=m, color="red", name="Points")
# m

In [ ]:
# certaines colonnes safran sont redondantes
safran = safran[safran.columns.difference(['dept','region2016','region'])]

if metrocom.codgeo.is_unique : print('Ok') 
else : print('ATTENTION codgeo ne doit pas servir d\'index !')
if safran.cell.is_unique : print('Ok') 
else : print('ATTENTION cell ne doit pas servir d\'index !')

# On joint les données des communes en left join aux données Agrosyst
spatialdf = gpd.sjoin(metrocom.set_geometry('centroid').set_index('codgeo'), safran.set_index('cell'), how='left', predicate='within')
# 357 communes sans maille rattaché par un within
join = spatialdf.loc[spatialdf.cell.isna(), ['geometry','centroid']]
join = gpd.sjoin_nearest(join.to_crs(3857).set_geometry('centroid'), safran.set_index('cell').to_crs(3857), how = 'left', distance_col = "distances").to_crs(4326)

spatialdf.loc[spatialdf.cell.isna(), :] = join[:]

# Joindre les domaines avec les données safran par le codeinsee
spatialdf['codeinsee'] = spatialdf.index
fdf = spatialdf.merge(fdf, on = 'codeinsee')

In [ ]:
# fdf['dlon'] = fdf['longitude']-fdf['centroid'].x
# fdf['dlat'] = fdf['latitude']-fdf['centroid'].y
# import plotly.express as px
# fig = px.histogram(fdf['dlon'], x="dlon", title="Distribution of delta longitude")
# fig.show()
# fig = px.histogram(fdf['dlat'], x="dlat", title="Distribution of delta latitude")
# fig.show()
# m = fdf.loc[(fdf['dlon'] > 0.08) | (fdf['dlat'] > 0.08)].set_geometry('geometry').explore(name="Polygons")
# m = fdf.loc[(fdf['dlon'] > 0.08) | (fdf['dlat'] > 0.08)].set_geometry('centroid').explore(m=m, color="red", name="Points")
# m


# fdf.set_index('domain_id').sample(25).to_csv(EXTERN_DATA_PATH+'test_safran_insee'+'.csv', index=True)